In [ ]:
# default_exp model.dummy

In [ ]:
# hide
%load_ext lab_black
%load_ext autoreload
%autoreload 2

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Dummy Model

> A small dummy model to test something quickly

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export
import torch
import torch.nn as nn
import torch.nn.functional as F

import pytorch_lightning as pl
import albumentations as A
import albumentations.pytorch.transforms as AT

from pytorch_lightning.metrics.functional import accuracy
from nbs_template.data.cifar import CIFAR10DataModule, CIFAR10

from loguru import logger

In [ ]:
# export
class DummyModel(pl.LightningModule):
    """DummyModel to test out something quickly

    Args:
        channels (`int`): the channels in the input image
        width (`int`): width of the image
        height (`int`): height of the image
        hidden_size (`int`): number of channels in the hidden layers
        learning_rate (`float`): the learning rate for the optimizer
    """

    def __init__(
        self, channels, width, height, num_classes, hidden_size=64, learning_rate=2e-4
    ):

        super().__init__()

        self.save_hyperparameters()

        # We take in input dimensions as parameters and use those to dynamically build model.
        self.channels = channels
        self.width = width
        self.height = height
        self.num_classes = num_classes
        self.hidden_size = hidden_size
        self.learning_rate = learning_rate

        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(channels * width * height, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_size, num_classes),
        )

    def forward(self, x):
        x = self.model(x)
        return F.log_softmax(x, dim=1)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        return {"loss": loss}

    def training_epoch_end(self, outputs):
        loss = torch.mean(torch.tensor([x["loss"] for x in outputs]))

        self.log("train_loss", loss)

    def validation_step(self, batch, batch_idx):

        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)

        return {"val_loss": loss, "val_acc": acc}

    def validation_epoch_end(self, outputs):

        avg_acc = torch.stack([x["val_acc"] for x in outputs]).mean()
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()

        self.log_dict({"val_loss": avg_loss, "val_acc": avg_acc})

        logger.info(f"val_loss: {avg_loss}, val_acc: {avg_acc}")

    def test_step(self, batch, batch_idx):
        x, y = batch
        preds = self(x)

        acc = accuracy(preds, y)

        return {"test_acc": acc}

    def test_epoch_end(self, outputs):
        avg_acc = torch.stack([x["test_acc"] for x in outputs]).mean()

        self.log("test_acc", avg_acc)

        logger.info(f"test_acc: {avg_acc}")

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

## Instantiating Normally

In [ ]:
transform = A.Compose(
    [
        A.Resize(32, 32),
        A.Normalize(mean=CIFAR10.mean, std=CIFAR10.std),
        AT.ToTensor(),
    ]
)
dm = CIFAR10DataModule(transform=transform)
model = DummyModel(*dm.size(), dm.num_classes, hidden_size=32)
trainer = pl.Trainer(max_epochs=1, progress_bar_refresh_rate=20)
trainer.fit(model, dm)

GPU available: False, used: False
INFO:lightning:GPU available: False, used: False
TPU available: None, using: 0 TPU cores
INFO:lightning:TPU available: None, using: 0 TPU cores


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified



  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 99.7 K
-------------------------------------
99.7 K    Trainable params
0         Non-trainable params
99.7 K    Total params
INFO:lightning:
  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 99.7 K
-------------------------------------
99.7 K    Trainable params
0         Non-trainable params
99.7 K    Total params
/home/shadowleaf/anaconda3/envs/dev-ink/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


2020-12-15 13:18:35.626 | INFO     | __main__:validation_epoch_end:72 - val_loss: 2.3227691650390625, val_acc: 0.09375
/home/shadowleaf/anaconda3/envs/dev-ink/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


2020-12-15 13:18:46.124 | INFO     | __main__:validation_epoch_end:72 - val_loss: 1.6776868104934692, val_acc: 0.4112261235713959


1

In [ ]:
trainer.test(model, datamodule=dm)

Files already downloaded and verified


/home/shadowleaf/anaconda3/envs/dev-ink/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


2020-12-15 13:18:48.150 | INFO     | __main__:test_epoch_end:87 - test_acc: 0.4146365821361542



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.4146),
 'val_acc': tensor(0.4112),
 'val_loss': tensor(1.6777)}
--------------------------------------------------------------------------------


[{'val_loss': 1.6776868104934692,
  'val_acc': 0.4112261235713959,
  'test_acc': 0.4146365821361542}]

## Instantiating using Hydra dataclass

In [ ]:
import hydra
from omegaconf import OmegaConf

from dataclasses import dataclass, field
from hydra.core.config_store import ConfigStore
from hydra.utils import instantiate
from hydra.experimental import (
    initialize,
    initialize_config_module,
    initialize_config_dir,
    compose,
)

In [ ]:
from typing import Dict, Any, List
from importlib import import_module

In [ ]:
@dataclass
class ModelConfig:
    _target_: str = "nbs_template.model.dummy.DummyModel"
    channels: int = 3
    width: int = 32
    height: int = 32
    num_classes: int = 10
    hidden_size: int = 32
    learning_rate: float = 2e-4

In [ ]:
@dataclass
class DataModuleConfig:
    type: str = "nbs_template.data.cifar.CIFAR10DataModule"
    transform: List[str] = field(
        default_factory=lambda: [
            "A.Resize(32, 32)",
            "A.Normalize(mean=CIFAR10.mean, std=CIFAR10.std)",
            "AT.ToTensor()",
        ]
    )

In [ ]:
@dataclass
class TrainingConfig:
    _target_: str = "pytorch_lightning.Trainer"
    max_epochs: int = 1
    progress_bar_refresh_rate: int = 20

In [ ]:
@dataclass
class DummyTrainingConfig:
    model_config: ModelConfig = ModelConfig
    dm_config: DataModuleConfig = DataModuleConfig
    trainer_config: TrainingConfig = TrainingConfig

In [ ]:
cs: ConfigStore = ConfigStore.instance()
cs.store(name="dummy_config", node=DummyTrainingConfig)

In [ ]:
with initialize(config_path="configs", job_name="app"):
    cfg = compose(config_name="dummy_config")

In [ ]:
cfg

{'model_config': {'_target_': 'nbs_template.model.dummy.DummyModel', 'channels': 3, 'width': 32, 'height': 32, 'num_classes': 10, 'hidden_size': 32, 'learning_rate': 0.0002}, 'dm_config': {'type': 'nbs_template.data.cifar.CIFAR10DataModule', 'transform': ['A.Resize(32, 32)', 'A.Normalize(mean=CIFAR10.mean, std=CIFAR10.std)', 'AT.ToTensor()']}, 'trainer_config': {'_target_': 'pytorch_lightning.Trainer', 'max_epochs': 1, 'progress_bar_refresh_rate': 20}}

In [ ]:
print(OmegaConf.to_yaml(cfg))

model_config:
  _target_: nbs_template.model.dummy.DummyModel
  channels: 3
  width: 32
  height: 32
  num_classes: 10
  hidden_size: 32
  learning_rate: 0.0002
dm_config:
  type: nbs_template.data.cifar.CIFAR10DataModule
  transform:
  - A.Resize(32, 32)
  - A.Normalize(mean=CIFAR10.mean, std=CIFAR10.std)
  - AT.ToTensor()
trainer_config:
  _target_: pytorch_lightning.Trainer
  max_epochs: 1
  progress_bar_refresh_rate: 20



In [ ]:
model = instantiate(cfg.model_config)
transforms = A.Compose([eval(x) for x in cfg.dm_config.transform])
dm_module_path, dm_class = cfg.dm_config.type.rsplit(".", 1)
dm_module = import_module(dm_module_path)
dm = getattr(dm_module, dm_class)(transform=transforms)
trainer = instantiate(cfg.trainer_config)

GPU available: False, used: False
TPU available: None, using: 0 TPU cores


In [ ]:
trainer.fit(model, dm)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified



  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 99.7 K
-------------------------------------
99.7 K    Trainable params
0         Non-trainable params
99.7 K    Total params
/home/shadowleaf/anaconda3/envs/dev-ink/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


2020-12-15 14:16:30.295 | INFO     | nbs_template.model.dummy:validation_epoch_end:90 - val_loss: 2.3173904418945312, val_acc: 0.125
/home/shadowleaf/anaconda3/envs/dev-ink/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


2020-12-15 14:16:40.879 | INFO     | nbs_template.model.dummy:validation_epoch_end:90 - val_loss: 1.6747769117355347, val_acc: 0.41401273012161255


1

In [ ]:
trainer.test(model, datamodule=dm)

Files already downloaded and verified


/home/shadowleaf/anaconda3/envs/dev-ink/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


2020-12-15 14:17:22.673 | INFO     | nbs_template.model.dummy:test_epoch_end:105 - test_acc: 0.41383785009384155



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.4138),
 'val_acc': tensor(0.4140),
 'val_loss': tensor(1.6748)}
--------------------------------------------------------------------------------


[{'val_loss': 1.6747769117355347,
  'val_acc': 0.41401273012161255,
  'test_acc': 0.41383785009384155}]

## Instantiating using Hydra config files

In [ ]:
%%file configs/dummy.yaml
model_config:
  _target_: nbs_template.model.dummy.DummyModel
  channels: 3
  width: 32
  height: 32
  num_classes: 10
  hidden_size: 32
  learning_rate: 0.0002
dm_config:
  type: nbs_template.data.cifar.CIFAR10DataModule
  transform:
  - A.Resize(32, 32)
  - A.Normalize(mean=CIFAR10.mean, std=CIFAR10.std)
  - AT.ToTensor()
trainer_config:
  _target_: pytorch_lightning.Trainer
  max_epochs: 1
  progress_bar_refresh_rate: 20

Writing configs/dummy.yaml


In [ ]:
with initialize(config_path="configs", job_name="app"):
    cfg = compose(config_name="dummy.yaml")

In [ ]:
cfg

{'model_config': {'_target_': 'nbs_template.model.dummy.DummyModel', 'channels': 3, 'width': 32, 'height': 32, 'num_classes': 10, 'hidden_size': 32, 'learning_rate': 0.0002}, 'dm_config': {'type': 'nbs_template.data.cifar.CIFAR10DataModule', 'transform': ['A.Resize(32, 32)', 'A.Normalize(mean=CIFAR10.mean, std=CIFAR10.std)', 'AT.ToTensor()']}, 'trainer_config': {'_target_': 'pytorch_lightning.Trainer', 'max_epochs': 1, 'progress_bar_refresh_rate': 20}}

In [ ]:
model = instantiate(cfg.model_config)
transforms = A.Compose([eval(x) for x in cfg.dm_config.transform])
dm_module_path, dm_class = cfg.dm_config.type.rsplit(".", 1)
dm_module = import_module(dm_module_path)
dm = getattr(dm_module, dm_class)(transform=transforms)
trainer = instantiate(cfg.trainer_config)

GPU available: False, used: False
TPU available: None, using: 0 TPU cores


In [ ]:
trainer.fit(model, dm)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified



  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 99.7 K
-------------------------------------
99.7 K    Trainable params
0         Non-trainable params
99.7 K    Total params
/home/shadowleaf/anaconda3/envs/dev-ink/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


2020-12-15 14:25:21.618 | INFO     | nbs_template.model.dummy:validation_epoch_end:90 - val_loss: 2.312533378601074, val_acc: 0.046875
/home/shadowleaf/anaconda3/envs/dev-ink/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


2020-12-15 14:25:31.941 | INFO     | nbs_template.model.dummy:validation_epoch_end:90 - val_loss: 1.7183785438537598, val_acc: 0.39729300141334534


1

In [ ]:
trainer.test(model, datamodule=dm)

Files already downloaded and verified


/home/shadowleaf/anaconda3/envs/dev-ink/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


2020-12-15 14:25:33.916 | INFO     | nbs_template.model.dummy:test_epoch_end:105 - test_acc: 0.4044528901576996



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.4045),
 'val_acc': tensor(0.3973),
 'val_loss': tensor(1.7184)}
--------------------------------------------------------------------------------


[{'val_loss': 1.7183785438537598,
  'val_acc': 0.39729300141334534,
  'test_acc': 0.4044528901576996}]

In [ ]:
from nbdev.export import notebook2script

notebook2script()

Converted 00_data.cifar.ipynb.
Converted 01_model.dummy.ipynb.
Converted index.ipynb.
Converted template_nb.ipynb.
